In [3]:
!pip install gym
!pip install stable_baselines3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
from gym import Env
from gym.spaces import Discrete, Box
import random
import numpy as np
import torch
from stable_baselines3 import SAC, PPO
import gym

In [21]:
### Define a two line manifold environment ###
class simEnv(Env):
    def __init__(self):
        # Action agent can take
        self.action_space = Box(np.array([-1, -1]), np.array([1, 1]))
        # Work space
        self.observation_space = Box(np.array([-10, -10, 0]), np.array([10, 10, 1]))
        # Initial starting point
        self.state = np.array([0.0, 0.0, 0])
        # Goal state
        self.goal = np.array([10.0, 10.0, 1])
        # Intersection point
        self.intersection = np.array([10.0, 0.0, 0])
        self.steps = 0
    
    def step(self, action):
        ### Projection ###
        self.steps += 1
        if self.state[2] == 1:
            _action = np.array([0.0, action[1]])
        else:
            _action = np.array([action[0], 0.0])
        self.state[0] += _action[0]
        self.state[1] += _action[1]
        if self.state[0] == 10:
            self.state[2] = 1
        done = False
        # Design rewards
        if self.state[1] != 0 and self.state[0] != 10: # Fall off the manifold
            # negative of l2 distance to current manifold reward
            if self.state[2] == 0:
                reward = -np.abs(self.state[1])
            else:
                reward = -np.abs(self.state[0] - 10)
            done = True
        elif np.array_equal(self.state, self.intersection): # Get to the intersection point
            if self.state[2] == 1:
                reward = -np.linalg.norm(self.state - self.intersection) / 2
            else:
                reward = 10
        elif np.array_equal(self.state, self.goal): # Reach the goal 
            reward = 100
            done = True
        else: # Direct the agent to intersection point / goal
            if self.state[2] == 1:
                reward = -np.linalg.norm(self.state - self.goal) / 2
#                 reward = 1 / np.linalg.norm(self.state - self.goal)
            else: 
                reward = -np.linalg.norm(self.state - self.intersection)
#                 reward = 1 / np.linalg.norm(self.state - self.intersection)
        # Set placeholder for info 
        info = {}
        if self.steps == 10:
            done = True
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.state = np.array([0.0,0.0, 0])
        return self.state

In [6]:
env = simEnv()

/home/fantasticoven/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
device = torch.device(
        "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = PPO('MlpPolicy', env, verbose=1, device=device, ent_coef=.15)
model = model.learn(total_timesteps=500000, eval_freq=1000)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1146 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 918          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0067785904 |
|    clip_fraction        | 0.039        |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.89        |
|    explained_variance   | 0.00126      |
|    learning_rate        | 0.0003       |
|    loss                 | 9.07e+03     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000775    |
|    

-----------------------------------------
| time/                   |             |
|    fps                  | 824         |
|    iterations           | 12          |
|    time_elapsed         | 29          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.019524831 |
|    clip_fraction        | 0.174       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.84       |
|    explained_variance   | 2.47e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.42        |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00612    |
|    std                  | 1.7         |
|    value_loss           | 11.3        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 815        |
|    iterations           | 13         |
|    time_elapsed         | 32        

-----------------------------------------
| time/                   |             |
|    fps                  | 808         |
|    iterations           | 23          |
|    time_elapsed         | 58          |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.006898463 |
|    clip_fraction        | 0.0911      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.34       |
|    explained_variance   | 1.98e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | -0.365      |
|    n_updates            | 220         |
|    policy_gradient_loss | -0.00284    |
|    std                  | 2.22        |
|    value_loss           | 0.706       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 810         |
|    iterations           | 24          |
|    time_elapsed         | 60    

-----------------------------------------
| time/                   |             |
|    fps                  | 825         |
|    iterations           | 34          |
|    time_elapsed         | 84          |
|    total_timesteps      | 69632       |
| train/                  |             |
|    approx_kl            | 0.018421888 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.7        |
|    explained_variance   | 0.00245     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.549      |
|    n_updates            | 330         |
|    policy_gradient_loss | 0.00115     |
|    std                  | 2.85        |
|    value_loss           | 0.345       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 826         |
|    iterations           | 35          |
|    time_elapsed         | 86    

-----------------------------------------
| time/                   |             |
|    fps                  | 835         |
|    iterations           | 45          |
|    time_elapsed         | 110         |
|    total_timesteps      | 92160       |
| train/                  |             |
|    approx_kl            | 0.016770612 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.16       |
|    explained_variance   | -0.000144   |
|    learning_rate        | 0.0003      |
|    loss                 | -0.675      |
|    n_updates            | 440         |
|    policy_gradient_loss | 0.00939     |
|    std                  | 3.94        |
|    value_loss           | 0.245       |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 836          |
|    iterations           | 46           |
|    time_elapsed         | 11

...

-----------------------------------------
| time/                   |             |
|    fps                  | 775         |
|    iterations           | 199         |
|    time_elapsed         | 525         |
|    total_timesteps      | 407552      |
| train/                  |             |
|    approx_kl            | 0.009450253 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss         | -11.9       |
|    explained_variance   | 0.00983     |
|    learning_rate        | 0.0003      |
|    loss                 | -1.66       |
|    n_updates            | 1980        |
|    policy_gradient_loss | 0.00763     |
|    std                  | 463         |
|    value_loss           | 0.211       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 775         |
|    iterations           | 200         |
|    time_elapsed         | 528   

------------------------------------------
| time/                   |              |
|    fps                  | 774          |
|    iterations           | 210          |
|    time_elapsed         | 555          |
|    total_timesteps      | 430080       |
| train/                  |              |
|    approx_kl            | 0.0052210228 |
|    clip_fraction        | 0.098        |
|    clip_range           | 0.2          |
|    entropy_loss         | -12.5        |
|    explained_variance   | 0.019        |
|    learning_rate        | 0.0003       |
|    loss                 | -1.78        |
|    n_updates            | 2090         |
|    policy_gradient_loss | 0.00705      |
|    std                  | 686          |
|    value_loss           | 0.17         |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 774         |
|    iterations           | 211         |
|    time_elaps

------------------------------------------
| time/                   |              |
|    fps                  | 775          |
|    iterations           | 221          |
|    time_elapsed         | 583          |
|    total_timesteps      | 452608       |
| train/                  |              |
|    approx_kl            | 0.0059260055 |
|    clip_fraction        | 0.119        |
|    clip_range           | 0.2          |
|    entropy_loss         | -13          |
|    explained_variance   | 0.00306      |
|    learning_rate        | 0.0003       |
|    loss                 | -1.87        |
|    n_updates            | 2200         |
|    policy_gradient_loss | 0.00744      |
|    std                  | 967          |
|    value_loss           | 0.107        |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 776         |
|    iterations           | 222         |
|    time_elaps

------------------------------------------
| time/                   |              |
|    fps                  | 780          |
|    iterations           | 232          |
|    time_elapsed         | 609          |
|    total_timesteps      | 475136       |
| train/                  |              |
|    approx_kl            | 0.0053694844 |
|    clip_fraction        | 0.065        |
|    clip_range           | 0.2          |
|    entropy_loss         | -13.5        |
|    explained_variance   | 1.54e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | -1.98        |
|    n_updates            | 2310         |
|    policy_gradient_loss | 0.0015       |
|    std                  | 1.4e+03      |
|    value_loss           | 0.163        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 780          |
|    iterations           | 233          |
|    time_e

-----------------------------------------
| time/                   |             |
|    fps                  | 782         |
|    iterations           | 243         |
|    time_elapsed         | 635         |
|    total_timesteps      | 497664      |
| train/                  |             |
|    approx_kl            | 0.005083856 |
|    clip_fraction        | 0.0874      |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.1       |
|    explained_variance   | 0.00149     |
|    learning_rate        | 0.0003      |
|    loss                 | -2.04       |
|    n_updates            | 2420        |
|    policy_gradient_loss | 0.00359     |
|    std                  | 1.99e+03    |
|    value_loss           | 0.121       |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 782        |
|    iterations           | 244        |
|    time_elapsed         | 638       

In [9]:
from stable_baselines3.common.evaluation import evaluate_policy

In [24]:
eval_env = simEnv()
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=1, deterministic=True)

In [25]:
print(mean_reward, std_reward)

-248092.51149368286 0.0


In [15]:
obs = env.reset()

for _ in range(100):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print('=====')
    print(obs)
    print(rewards)

=====
[1. 0. 0.]
-9.0
=====
[2. 0. 0.]
-8.0
=====
[3. 0. 0.]
-7.0
=====
[4. 0. 0.]
-6.0
=====
[5. 0. 0.]
-5.0
=====
[6. 0. 0.]
-4.0
=====
[7. 0. 0.]
-3.0
=====
[8. 0. 0.]
-2.0
=====
[9. 0. 0.]
-1.0
=====
[10.  0.  1.]
-5.0
=====
[10. -1.  1.]
-5.5
=====
[10. -2.  1.]
-6.0
=====
[10. -1.  1.]
-5.5
=====
[10.  0.  1.]
-5.0
=====
[10.  1.  1.]
-4.5
=====
[10.  0.  1.]
-5.0
=====
[10. -1.  1.]
-5.5
=====
[10.  0.  1.]
-5.0
=====
[10.  1.  1.]
-4.5
=====
[10.  0.  1.]
-5.0
=====
[10. -1.  1.]
-5.5
=====
[10.  0.  1.]
-5.0
=====
[10.  1.  1.]
-4.5
=====
[10.  2.  1.]
-4.0
=====
[10.  3.  1.]
-3.5
=====
[10.  4.  1.]
-3.0
=====
[10.  5.  1.]
-2.5
=====
[10.  6.  1.]
-2.0
=====
[10.  5.  1.]
-2.5
=====
[10.  4.  1.]
-3.0
=====
[10.  3.  1.]
-3.5
=====
[10.  4.  1.]
-3.0
=====
[10.  3.  1.]
-3.5
=====
[10.  2.  1.]
-4.0
=====
[10.  3.  1.]
-3.5
=====
[10.  4.  1.]
-3.0
=====
[10.  5.  1.]
-2.5
=====
[10.  6.  1.]
-2.0
=====
[10.  5.  1.]
-2.5
=====
[10.  6.  1.]
-2.0
=====
[10.  7.  1.]
-1.5
==

In [ ]:
# env = simEnv()

# device = torch.device(
#         "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# model = SAC('MlpPolicy', env, verbose=1, device=device)
# model = model.learn(total_timesteps=500000, eval_freq=1000)

In [ ]:
# obs = env.reset()

# for _ in range(100):
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     print(obs)
#     print(rewards)